<a href="https://colab.research.google.com/github/BruceAuHK/Dashboard_FFO/blob/master/Plotly_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import urllib.request
import json
import ssl
import plotly as py
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
from scipy import special 
import pandas as pd
from pandas.io.json import json_normalize
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

context = ssl._create_unverified_context()

def get_data_from_HKMA(url):
    # Balance sheet
    url = url

    # retrieve data from hkma api, 100 entries per call, untill every data are retrieveed
    records = []
    offset = 0
    while True:
        with urllib.request.urlopen(url+"&offset="+str(offset), context=context) as req:
            data = json.loads(req.read().decode())
            records += data['result']['records']
        if data['result']['datasize'] <= 0:
            break
        else:
            offset += 100

    # now data retrieved from hkma are stored in 'records' in a chronological order
    return records


print("dependencies ready")


dependencies ready


# **Monetary Base**

In [10]:
MBC = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/monetary-operation/monetary-base-endperiod?choose=end_of_month&from=1998-09&to=now&sortby=end_of_month&sortorder=asc'

#1. Size of the Monetary Base was published since 25 November 1998.
mbc_df = pd.json_normalize(get_data_from_HKMA(MBC))

#Convert dataframe to dataframe long format
mbc_dfl = pd.melt(mbc_df, id_vars = ['end_of_month'], value_vars =['cert_of_indebt', 'gov_notes_coins_circulation' , 'aggr_balance_bf_disc_win' , 'outstanding_efbn' , 'ow_lb_bf_disc_win' , 'mb_bf_disc_win_total'],var_name='Size of the Monetary Base', value_name='HK$ million' )

fig = px.area(mbc_dfl, x='end_of_month' , y='HK$ million', color = 'Size of the Monetary Base')

fig.show()
